In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import pickle as pkl

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
print('all imported')

all imported


In [3]:
df = pd.read_csv('../bucket/Readmission_30days/Readmission/Demo_Readmission_label.csv') #contains gender, race, ethnicity, DoB
df.head(2)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,PATIENT_DK,ADMISSION_DTM,DISCHARGE_DTM,CLOSEST_READMISSION_DTM,Gap in months,VISIT_NUMBER,#,PATIENT_BIRTH_DATE,PATIENT_DEATH_DATE,PATIENT_DECEASED_FLAG,PATIENT_ETHNICITY_NAME,PATIENT_GENDER_NAME,PATIENT_PRIMARY_CITY,PATIENT_RACE_NAME,PATIENT_RELIGION_NAME
0,0,0,9179017,2019-02-07 13:36:00,2019-02-13 14:44:00.000000,2019-04-25 07:06:00,2.0,2000266379652,443153.0,1966-10-25 00:00:00,NaN,N,Not Hispanic or Latino,MALE,CHARLES CITY,White,Church of Christ A Cappella
1,1,1,1798150,2019-03-06 06:47:00,2019-03-06 08:33:00.000000,2019-07-29 17:26:00,4.0,2000270215822,68380.0,1948-11-29 00:00:00,NaN,N,Not Hispanic or Latino,MALE,DODGE CENTER,White,Lutheran


In [87]:
#RACE Preparation
races = df.PATIENT_RACE_NAME.unique()
df['PATIENT_RACE_NAME'] = df['PATIENT_RACE_NAME'].replace({'Unknown':'UNKNOWN', 'Other': 'UNKNOWN', 
                                                   'Choose Not to Disclose': 'UNKNOWN',
                                                   'Unable to Provide': 'UNKNOWN', 
                                                          ' ': 'UNKNOWN'})
for r in races:
    if type(r) == str and r.startswith('Asian'):
        df['PATIENT_RACE_NAME'] = df['PATIENT_RACE_NAME'].replace({r:'Asian'})

for r in races:
    if type(r) == str and ('black' in r.lower() or 'african' in r .lower()):
        df['PATIENT_RACE_NAME'] = df['PATIENT_RACE_NAME'].replace({r:'Black'})
        
for r in races:
    if type(r) == str and ('american indian' in r.lower() or 'alaskan' in r .lower()):
        df['PATIENT_RACE_NAME'] = df['PATIENT_RACE_NAME'].replace({r:'American Indian/Alaskan Native'})

        
for r in races:
    if type(r) == str and ('hawaii' in r.lower() or 'pacific' in r .lower() or 'samoan' in r.lower() or 'guam' in r.lower()):
        df['PATIENT_RACE_NAME'] = df['PATIENT_RACE_NAME'].replace({r:'Native Hawaiian/Pacific Islander'})

df = df.fillna(value={'PATIENT_RACE_NAME':'UNKNOWN'}).copy()
df.PATIENT_RACE_NAME.value_counts()

White                               270925
UNKNOWN                              30831
Black                                10415
Asian                                 5131
American Indian/Alaskan Native        1683
Native Hawaiian/Pacific Islander       368
Name: PATIENT_RACE_NAME, dtype: int64

In [88]:
#ETHNIC Preparation
races = df.PATIENT_ETHNICITY_NAME.unique()
df['PATIENT_ETHNICITY_NAME'] = df['PATIENT_ETHNICITY_NAME'].replace({'Unknown':'UNKNOWN', 'Other': 'UNKNOWN', 
                                                   'Choose Not to Disclose': 'UNKNOWN',
                                                   'Unable to Provide': 'UNKNOWN'})

for r in races:
    if type(r) == str and ('cuba' in r.lower() or 'mexic' in r.lower() or 'puerto' in r .lower() or 'central americ' in r.lower() or 'south americ' in r.lower() or 'spanish' in r.lower()):
        df['PATIENT_ETHNICITY_NAME'] = df['PATIENT_ETHNICITY_NAME'].replace({r:'Hispanic or Latino'})

df = df.fillna(value={'PATIENT_ETHNICITY_NAME':'UNKNOWN'}).copy()       
df.PATIENT_ETHNICITY_NAME.value_counts()

Not Hispanic or Latino    277976
UNKNOWN                    30079
Hispanic or Latino         11298
Name: PATIENT_ETHNICITY_NAME, dtype: int64

In [89]:
#AGE Prep
df.ADMISSION_DTM = pd.to_datetime(df.ADMISSION_DTM, errors='coerce')
df.DISCHARGE_DTM = pd.to_datetime(df.DISCHARGE_DTM, errors='coerce')
df.PATIENT_BIRTH_DATE = pd.to_datetime(df.PATIENT_BIRTH_DATE, errors='coerce')

df['PATIENT_AGE'] = (df['ADMISSION_DTM'] - df['PATIENT_BIRTH_DATE']).astype('<m8[Y]')

def to_bins(x):
    if np.isnan(x):
        return -1
    else:
        if x<100:
            return int(x/10)
        else:
            return 10
df['PATIENT_AGE_BINNED'] = df['PATIENT_AGE'].apply(to_bins)
df['PATIENT_AGE_BINNED'].value_counts()

 6     72961
 7     65096
 5     49205
 8     32825
 4     27411
 3     25515
 2     20125
 0      8648
 1      8260
 9      8205
 10      954
-1       148
Name: PATIENT_AGE_BINNED, dtype: int64

In [90]:
df.to_csv('Readmission_Data/Readmission_label_Demo_processed.csv')

In [81]:
df['INVALID'] = (df.ADMISSION_DTM.isnull()) | (df.DISCHARGE_DTM.isnull())
df = df.loc[df.INVALID==False].copy()

df_train = df.loc[df.PATIENT_DK.isin(dct['patients_train'])]
df_test = df.loc[df.PATIENT_DK.isin(dct['patients_test'])]
len(df_train), len(df_test)

(230369, 63280)

In [130]:
len(temp)

27838

## recomputation of age

In [21]:
df = pd.read_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_new.csv')
len(df)

27838

In [7]:
for c in df.columns:
    if 'birth' in c.lower():
        print(c)

PATIENT_BIRTH_DATE
Delivery/Birthing Room Attendance and Resuscitation Services


In [10]:
df.PATIENT_BIRTH_DATE.values[:10], df.ADMISSION_DTM.values[:10]

(array(['1939-08-21 00:00:00', '1950-05-25 00:00:00',
        '1948-12-16 00:00:00', '1947-04-26 00:00:00',
        '1957-04-02 00:00:00', '2018-08-06 00:00:00',
        '1930-07-27 00:00:00', '1958-12-09 00:00:00',
        '1963-11-20 00:00:00', '1937-09-17 00:00:00'], dtype=object),
 array(['2019-01-01 01:21:00', '2019-01-01 01:49:00',
        '2019-01-01 02:25:00', '2019-01-01 03:39:00',
        '2019-01-01 03:29:00', '2019-01-01 05:00:00',
        '2019-01-01 05:50:00', '2019-01-01 06:07:00',
        '2019-01-01 05:51:00', '2019-01-01 07:16:00'], dtype=object))

In [12]:
df['ADMISSION_DTM'] = pd.to_datetime(df['ADMISSION_DTM'] ,format = '%Y-%m-%d', errors = 'coerce')
df['PATIENT_BIRTH_DATE'] = pd.to_datetime(df['PATIENT_BIRTH_DATE'],format = '%Y-%m-%d', errors = 'coerce')


In [15]:
df.at[df.index[5], 'PATIENT_BIRTH_DATE']

Timestamp('2018-08-06 00:00:00')

In [18]:

age = df['ADMISSION_DTM'] - df['PATIENT_BIRTH_DATE']
years = [np.round(a.days/365.25) for a in age]
np.min(years), np.max(years)

(0.0, 170.0)

In [26]:
idx = [i for i in range(len(years)) if years[i]>100]
df['PATIENT_BIRTH_DATE'].values[idx], df['PATIENT_DK_x'].values[idx]

(array(['1850-01-01 00:00:00', '1850-01-01 00:00:00',
        '1850-01-01 00:00:00', '1850-01-01 00:00:00',
        '1915-04-27 00:00:00', '1850-01-01 00:00:00',
        '1918-07-20 00:00:00', '1918-01-05 00:00:00',
        '1918-07-20 00:00:00', '1916-06-26 00:00:00',
        '1850-01-01 00:00:00', '1850-01-01 00:00:00',
        '1917-04-02 00:00:00', '1850-01-01 00:00:00',
        '1850-01-01 00:00:00', '1850-01-01 00:00:00',
        '1901-01-01 00:00:00', '1917-10-31 00:00:00',
        '1850-01-01 00:00:00', '1901-01-01 00:00:00',
        '1917-10-21 00:00:00', '1917-05-20 00:00:00',
        '1901-01-01 00:00:00', '1918-03-24 00:00:00',
        '1850-01-01 00:00:00', '1918-07-20 00:00:00',
        '1850-01-01 00:00:00', '1901-01-01 00:00:00',
        '1850-01-01 00:00:00', '1901-01-01 00:00:00',
        '1901-01-01 00:00:00', '1901-01-01 00:00:00',
        '1901-01-01 00:00:00', '1915-09-05 00:00:00',
        '1850-01-01 00:00:00', '1917-09-01 00:00:00',
        '1901-01-01 00:00:00

In [30]:
df2 =  pd.read_csv('../bucket1/Readmission/EDTWH_DIM_PATIENT_HISTORY_3.csv')
df2.head(), df2.columns

(   #  PATIENT_DK PATIENT_BIRTH_DATE PATIENT_DEATH_DATE PATIENT_DECEASED_FLAG  \
 0  1       16720         1922-11-26         0001-01-01                     N   
 1  2       16720         1922-11-26                NaN                     N   
 2  3       16720         1922-11-26         0001-01-01                     N   
 3  4       16720         1922-11-26         2020-11-28                     Y   
 4  5       16720         1922-11-26                NaN                     N   
 
    PATIENT_ETHNICITY_NAME PATIENT_GENDER_NAME PATIENT_PRIMARY_CITY  \
 0  Not Hispanic or Latino              FEMALE            Rochester   
 1  Not Hispanic or Latino              FEMALE               WASECA   
 2  Not Hispanic or Latino              FEMALE               Waseca   
 3  Not Hispanic or Latino              FEMALE               Waseca   
 4  Not Hispanic or Latino              FEMALE               Waseca   
 
   PATIENT_RACE_NAME   PATIENT_RELIGION_NAME  
 0             White    CHRISTIAN, NO

In [34]:
print(len(df2))
df2 = df2.drop_duplicates(subset=['PATIENT_DK', 'PATIENT_BIRTH_DATE'])
len(df2)

302473


147063

In [35]:
pids = df['PATIENT_DK_x'].values[idx]
df
temp = df2.loc[df2.PATIENT_DK.isin(pids)]
temp.PATIENT_BIRTH_DATE.values, len(pids), len(temp)

(array(['1915-06-20', '1917-10-21', '1918-08-31', '1917-09-02',
        '1917-05-20', '1917-06-30', '1918-03-24', '1915-04-27',
        '1917-10-31', '1916-06-26', '1917-03-25', '1918-06-27',
        '1917-10-08', '1918-01-09', '1918-01-05', '1918-07-20',
        '1917-04-02', '1917-08-17', '1915-10-31', '1916-02-02',
        '1917-09-14', '1917-07-28', '1917-02-02', '1917-04-17',
        '1918-01-10', '1918-10-06', '1917-11-03', '1850-01-01',
        '1958-06-22', '1917-09-01', '1918-07-26', '1850-01-01',
        '2017-12-01', '1850-01-01', '2018-02-07', '1850-01-01',
        '2018-03-08', '1850-01-01', '2018-05-16', '1850-01-01',
        '2018-06-21', '1850-01-01', '2018-07-04', '1850-01-01',
        '2018-07-30', '1850-01-01', '2018-08-25', '1850-01-01',
        '2018-08-31', '1850-01-01', '2018-10-15', '1850-01-01',
        '2018-10-17', '1915-09-05', '1850-01-01', '2018-11-02',
        '1850-01-01', '2018-11-13', '1850-01-01', '2018-12-04',
        '1850-01-01', '2018-12-04', '185

In [40]:
for i,j in df.iterrows():
    pid = df.at[i, 'PATIENT_DK_x']
    temp = df2.loc[df2.PATIENT_DK==pid]
    df.at[i, 'PATIENT_BIRTH_DATE'] = temp.at[temp.index[-1], 'PATIENT_BIRTH_DATE']
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [43]:
df['ADMISSION_DTM'] = pd.to_datetime(df['ADMISSION_DTM'] ,format = '%Y-%m-%d', errors = 'coerce')
# df['PATIENT_BIRTH_DATE'] = pd.to_datetime(df['PATIENT_BIRTH_DATE'],format = '%Y-%m-%d', errors = 'coerce')
# df['PATIENT_BIRTH_DATE']

In [44]:
age = df['ADMISSION_DTM'] - df['PATIENT_BIRTH_DATE']
years = [np.round(a.days/365.25) for a in age]
np.min(years), np.max(years)

(0.0, 104.0)

In [45]:
df['PATIENT_AGE_NEW'] = years

In [46]:
df.to_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_new.csv')


In [47]:
for c in df.columns:
    if 'AGE' in c or 'BIRTH' in c:
        print(c)

PATIENT_BIRTH_DATE
PATIENT_AGE
PATIENT_AGE_BINNED
PATIENT_AGE_NEW


In [ ]:
df=pd.read_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_new.csv')


In [49]:
df3 = pd.read_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_anonymized_new.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
len(df), len(df3)

(27838, 27838)

In [51]:
df3['PATIENT_AGE_NEW'] = df['PATIENT_AGE_NEW'].copy()

In [54]:
df3.to_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_anonymized_new_age.csv')

In [55]:
df3 = pd.read_csv('../bucket1/Amara/Readmission_Data/Baseline/Readmission_label_Demo_processed_w_xray_loc_expanded_w_MRN_48h_cpt_icd_DischargeNote_from_dnotes_file_w_DiagnosisNewL3_w_ClassID_w_XrayIndicator_w_LabCount_w_Death_anonymized_new_age.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
len(df3), 
temp3 = df3.loc[df3.PATIENT_AGE_NEW<5]
print(len(temp3)), temp3['Gap in months'].value_counts(), len(temp3.loc[temp3['Gap in months']==1])/len(temp3)

292


(None,
 1.0    34
 2.0    19
 3.0    12
 4.0     8
 5.0     3
 6.0     2
 Name: Gap in months, dtype: int64,
 0.11643835616438356)

In [67]:
'INCLUDED' in df3.columns

True